In [1]:
import numpy as np
import time
import pandas as pd

from game_functions.environment import sbfeatures as sbf
from game_functions.model import brain

Using Theano backend.
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Failed to import scipy.linalg.blas, and Theano flag blas.ldflags is empty. Falling back on slower implementations for dot(matrix, vector), dot(vector, matrix) and dot(vector, vector) (DLL load failed: %1 is not a valid Win32 application.)


In [2]:
# initalize environment
env = sbf.Environment()

init


In [3]:
# initialize controller
env.controller(initialize=True, close=True)
env.controller(initialize=True, opens=True)
env.action(5, delay=0.1)

5

In [9]:
# launch game
env.make()

launching game...


In [12]:
# start game from menu
env.start_game(botsOn=True, mode='easy')

In [4]:
# attached onto game memory
env.initalize_memory()

hwnd: 12454096
pid: c_long(8604)


In [5]:
# test observation
print(env.observation())
#env.__grab_addresses__()
#env.__grab_features__()

[-0.9, -4.5, -10.1, -5.7, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1]


In [6]:
# test observation
print(env.p1_score, env.p2_score)

0 0


In [16]:
# setup reward structure
env.reward_structure(p1_death=-1, p2_death=1, p1_damage=-0.01, p2_damage=0.01)

Reward structure defined!


In [17]:
# test reward calculation between states
print(env.reward(stale_observation=env.features, new_observation=env.observation()))

0


In [18]:
# test controller input
env.action(5, delay=0.1)

5

In [19]:
# define model

In [20]:
input_shape = 17


Qas = brain.keras_model(inputs=input_shape, dropouts=0.2, opti=1e-6, clipping=1, clipval=10)
Qas.load_weights('model_weights//kill_rewards_TD5_LA_LRELU//1500000_model_backup_20170628_0818.keras')

In [22]:
import time
import random
import datetime

episode = 0
training_backup = 500000
save_location = "training_data\\"
training_batch = []

# action settings
randomness = True

#score keeping
p1_score = 0
p2_score = 0

#init#
prev_observation = env.observation()[:]
observation = env.observation()[:]
header = ['episode','p1h', 'p1_x',' p1_y', 'p2h', 'p2_x',' p2_y',
          'right_of', 'left_of', 'above', 'below', 'x_clear', 'y_clear',
          'cd1', 'cd2',
         'aim_down', 'aim_left', 'aim_up', 'aim_right',
         'p1_x_delta', 'p2_x_delta', 'p1_y_delta', 'p2_y_delta',
         'action', 'reward', 'p1_score', 'p2_score']

while True:

        episode += 1
        
        #random action
        if randomness:
            action = random.randint(0, 7)

        #env.action(action, delay=0.1)
        
        observation = env.observation()[:]
        reward = env.reward(stale_observation=prev_observation, 
                            new_observation=observation)
        prev_observation = observation
        
        #datapoint = [episode]
        datapoint = []
        datapoint.extend(observation)
        

        
        
        predictions = env.create_predictions(datapoint)
        action_rewards = Qas.predict(predictions)
        
        #---action = np.where(advantage == np.amax(advantage))[0][0]
        action = np.where(action_rewards == np.amax(action_rewards))[0][0] # greedy action
        print(action, env.p1_score, env.p2_score)
        
        env.action(action, delay=0.1)
        
        #datapoint.append(action)
        #datapoint.append(reward)
        #datapoint.extend([env.p1_score + 25000, env.p2_score + 25000])

        training_batch.append(datapoint)
        
        if (len(training_batch) % training_backup) == 0:
            name =  save_location + str(episode) + '_training_backup_' + datetime.datetime.today().strftime('%Y%m%d_%H%M') + '.csv'
            exp = pd.DataFrame(training_batch)
            exp.columns = header
            exp.to_csv(name, index=False)
            print('Saving Training Experience...')
            training_batch = []
    
;

6 0 0
0 0 0
7 0 0
7 0 0
0 0 0
3 0 0
7 0 0
7 0 0
0 0 0
7 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
4 0 0
4 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0
0 0 0


KeyboardInterrupt: 